In [2]:
import os
import sys
print(os.getcwd())
sys.path.append("../WanPy")

from WanPy import *
from pythtb import *
from pythTB_wan import *
import models
import plotting as plot

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

/Users/treycole/Codes/WanPy/tutorials


Setting up `pythTB` tight-binding model

In [131]:
delta = 1
t0 = 0.4
tprime = 0.5

n_super_cell = 2
model = models.chessboard(t0, tprime, delta).make_supercell([[n_super_cell, 0], [0, n_super_cell]])

low_E_sites = np.arange(0, model.get_num_orbitals(), 2)
high_E_sites = np.arange(1, model.get_num_orbitals(), 2)
lat_vecs = model.get_lat()
orb_vecs = model.get_orb()

n_orb = model.get_num_orbitals()
n_occ = int(n_orb/2)

u_wfs_full = wf_array(model, [20, 20])
u_wfs_full.solve_on_grid([0, 0])
chern = u_wfs_full.berry_flux([i for i in range(n_occ)])/(2*np.pi)

print(f"Low energy sites: {low_E_sites}")
print(f"High energy sites: {high_E_sites}")
print(f"Chern #: {chern: .1f}")

Low energy sites: [0 2 4 6]
High energy sites: [1 3 5 7]
Chern #: -1.0


To store the eigenstates at their respective k-points, we will initialize the `Bloch` class that is effectively a wrapper to the `pythTB` `wf_array` class. A `Bloch` object takes as its first argument a `pythTB` `model`, followed by the number of k-points along each reciprocal lattice basis vector.

In [132]:
eigstates = Bloch(model, 4, 4)

The `Bloch` object has as an attribute the `K_mesh`. This class stores the regular array of k-points, along with methods that utilize the k-mesh when Wannierizing for example, on which the `Bloch` object will store the associated Bloch-like states. To see the k-mesh points in reduced units, we will call `K_mesh.full_mesh`.

In [133]:
eigstates.K_mesh.full_mesh

array([[[0.  , 0.  ],
        [0.  , 0.25],
        [0.  , 0.5 ],
        [0.  , 0.75]],

       [[0.25, 0.  ],
        [0.25, 0.25],
        [0.25, 0.5 ],
        [0.25, 0.75]],

       [[0.5 , 0.  ],
        [0.5 , 0.25],
        [0.5 , 0.5 ],
        [0.5 , 0.75]],

       [[0.75, 0.  ],
        [0.75, 0.25],
        [0.75, 0.5 ],
        [0.75, 0.75]]])

To initialize the Bloch energy eigenstates, along with the associated cell-periodic eigenstates and eigenenergies, as class attributes, we will call the `solve_model` method.

In [138]:
eigstates.solve_model()
energies = eigstates.get_energies()
unk_states = eigstates.get_states()["Cell periodic"]
# print(energies)

In [139]:
nan = np.empty(unk_states.shape)
nan.fill(np.NaN)

outer_window = [-2.5, 0]
inner_window = [-2, -1]

outer_states_sliced = np.where(
    np.logical_and(energies[..., np.newaxis] > outer_window[0], energies[..., np.newaxis] < outer_window[1]), unk_states, nan)
outer_mask = np.isnan(outer_states_sliced)
outer_masked_states = np.ma.masked_array(unk_states, mask=outer_mask)

inner_states_sliced = np.where(
    np.logical_and(energies[..., np.newaxis] > inner_window[0], energies[..., np.newaxis] < inner_window[1]), unk_states, nan)
inner_mask = np.isnan(inner_states_sliced)
inner_masked_states = np.ma.masked_array(unk_states, mask=inner_mask)

min_mask = ~np.logical_and(~outer_mask, inner_mask)
min_masked_states = np.ma.masked_array(unk_states, mask=min_mask)

In [140]:
# outer_masked_states

In [141]:
# inner_masked_states

In [142]:
# min_masked_states

In [145]:
N_outer = (~outer_masked_states.mask).sum(axis=(-1,-2))//n_orb
N_inner = (~inner_masked_states.mask).sum(axis=(-1,-2))//n_orb
N_min = (~min_masked_states.mask).sum(axis=(-1,-2))//n_orb
print(N_outer, "\n\n", N_inner, "\n\n", N_min)
# N_outer = N_inner + N_min

[[3 4 4 4]
 [4 4 4 4]
 [4 4 4 4]
 [4 4 4 4]] 

 [[2 2 2 2]
 [2 2 4 2]
 [2 4 4 4]
 [2 2 4 2]] 

 [[1 2 2 2]
 [2 2 0 2]
 [2 0 0 0]
 [2 2 0 2]]


In [130]:
P = np.random.rand(2, 2)
Z = min_masked_states.conj() @ P @ np.transpose(min_masked_states, axes=(0,1,3,2))
_, eigvecs = np.linalg.eigh(Z) # [val, idx]
eigvecs

masked_array(
  data=[[[[--, --],
          [--, --]],

         [[(-0.6467306973977949+0j), --],
          [--, --]],

         [[--, --],
          [--, --]],

         [[(-0.6467306973977949+0j), --],
          [--, --]]],


        [[[(-0.7762615783448653+0j), --],
          [--, --]],

         [[--, --],
          [--, --]],

         [[--, --],
          [--, --]],

         [[--, --],
          [--, --]]],


        [[[--, --],
          [--, --]],

         [[--, --],
          [--, --]],

         [[(0.3965078175638906+0j), --],
          [--, --]],

         [[--, --],
          [--, --]]],


        [[[(-0.7762615783448655+0j), --],
          [--, --]],

         [[--, --],
          [--, --]],

         [[--, --],
          [--, --]],

         [[--, --],
          [--, --]]]],
  mask=[[[[ True,  True],
          [ True,  True]],

         [[False,  True],
          [ True,  True]],

         [[ True,  True],
          [ True,  True]],

         [[False,  True],
          

In [129]:
N_wfs - N_inner

array([[1, 1, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0]])

In [128]:
N_wfs = 1
states_min = np.einsum('...ij, ...ik->...jk', eigvecs[..., -(N_wfs-N_inner):], min_masked_states)

TypeError: only integer scalar arrays can be converted to a scalar index

In [30]:
np.concatenate((min_masked_states, inner_masked_states), axis=-2).shape

(4, 4, 4, 2)

In [31]:
np.concatenate((min_masked_states, inner_masked_states), axis=-2)


masked_array(
  data=[[[[ 1.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j],
          [ 0.00000000e+00+0.00000000e+00j,
            1.00000000e+00+0.00000000e+00j],
          [ 1.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j],
          [ 0.00000000e+00+0.00000000e+00j,
            1.00000000e+00+0.00000000e+00j]],

         [[-9.68028222e-01+0.00000000e+00j,
           -1.77371590e-01+1.77371590e-01j],
          [ 2.50841308e-01+0.00000000e+00j,
           -6.84499320e-01+6.84499320e-01j],
          [-9.68028222e-01+0.00000000e+00j,
           -1.77371590e-01+1.77371590e-01j],
          [ 2.50841308e-01+0.00000000e+00j,
           -6.84499320e-01+6.84499320e-01j]],

         [[-8.88073834e-01+0.00000000e+00j,
           -3.25057584e-01+3.25057584e-01j],
          [-4.59700843e-01+0.00000000e+00j,
            6.27963030e-01-6.27963030e-01j],
          [-8.88073834e-01+0.00000000e+00j,
           -3.25057584e-01+3.25057584e-01j],
   

In [23]:
mask1 = [True, True, False, False, False]
mask2 = [False, True, True, False, False]

x = [1, 2, 3, 4, 5]

states1 = np.ma.masked_array(x, mask=mask1)
states2 = np.ma.masked_array(x, mask=mask2)

states1

masked_array(data=[--, --, 3, 4, 5],
             mask=[ True,  True, False, False, False],
       fill_value=999999)

In [24]:
states2

masked_array(data=[1, --, --, 4, 5],
             mask=[False,  True,  True, False, False],
       fill_value=999999)

In [26]:
np.concatenate((states1, states2))

masked_array(data=[1, 2, 3, 4, 5, 1, 2, 3, 4, 5],
             mask=False,
       fill_value=999999)

In [ ]:
N_wfs = 

In [ ]:
energies = self.energy_eigstates.get_energies()
unk_states = self.energy_eigstates.get_states()["Cell periodic"]
tilde_states = self.tilde_states.get_states()["Cell periodic"]

# number of states in target manifold 
if N_wfs is None:
    N_wfs = self.tilde_states._n_states

# outer window
if outer_window == "occupied":
    outer_window_type = "bands"
    outer_band_idxs = list(range(n_occ))
    outer_band_energies = energies[..., :n_occ]
    outer_window = [np.argmin(outer_band_energies), np.argmax(outer_band_energies)]
    outer_states = unk_states.take(list(range(n_occ)), axis=-2)
elif list(outer_window.keys())[0].lower() == 'bands':
    outer_window_type = "bands"
    outer_band_idxs = list(outer_window.vals())[0]
    outer_band_energies = energies[..., outer_band_idxs]
    outer_window = [np.argmin(outer_band_energies), np.argmax(outer_band_energies)]
    outer_states = unk_states.take(outer_band_idxs, axis=-2)
    N_outer = len(outer_band_idxs)
elif list(outer_window.keys())[0].lower() == 'energy':
    outer_window_type = "energy"
    outer_window = np.sort(list(outer_window.vals())[0])
    nan = np.empty(unk_states.shape)
    nan.fill(np.NaN)
    states_sliced = np.where(
        np.logical_and(
            energies[..., np.newaxis] > outer_window[0], energies[..., np.newaxis] < outer_window[1]), unk_states, nan
            )
    mask_outer = np.isnan(states_sliced)
    masked_outer_states = np.ma.masked_array(states_sliced, mask=mask_outer)
    outer_states = masked_outer_states

# inner window
if inner_window is None:
    N_inner = 0
elif list(inner_window.keys())[0].lower() == 'bands':
    inner_window_type = "bands"
    inner_band_idxs = list(inner_window.vals())[0]
    inner_band_energies = energies[..., inner_band_idxs]
    inner_window = [np.argmin(inner_band_energies), np.argmax(inner_band_energies)]
    inner_states = unk_states.take(inner_band_idxs, axis=-2)
    N_inner = len(inner_band_idxs)
elif list(inner_window.keys())[0].lower() == 'energy':
    inner_window_type = "energy"
    inner_window =  np.sort(list(outer_window.vals())[0])
    nan = np.empty(unk_states.shape)
    nan.fill(np.NaN)
    states_sliced = np.where(
        np.logical_and(
            energies[..., np.newaxis] > inner_window[0], energies[..., np.newaxis] < inner_window[1]), unk_states, nan
            )
    mask_inner = np.isnan(states_sliced)
    masked_inner_states = np.ma.masked_array(states_sliced, mask=mask_inner)
    inner_states = masked_inner_states

# minimization manifold
if inner_window is not None:
    if inner_window_type == 'energy' and outer_window_type == 'energy':
        min_mask = ~np.logical_and(~mask_outer, mask_inner)
        min_states = np.ma.masked_array(unk_states, mask=min_mask)
    elif inner_window_type == 'bands' and outer_window_type == 'bands':
        min_band_idxs = list(np.setdiff1d(outer_band_idxs, inner_band_idxs))
        min_states = unk_states.take(min_band_idxs, axis=-2)
else:
    min_states = outer_states